In [11]:
import os
import json
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [12]:
root = os.getcwd()

gauth = GoogleAuth()       
gauth.DEFAULT_SETTINGS['client_config_file'] = f'{root}/tmp_data/client_secrets.json'
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

key_id = '1OeHp-PDisvt3Et6CzlUK6J5o8d_Omm9R'
#root = "C:/Users/Miguel/virgo_airflow/tmp_data/"
filename = '/tmp_data/stocks_dashboard.html'

upload_file= f'{root}{filename}'


gfile = drive.CreateFile(
    {
        'title': filename,
        'parents': [{'id': key_id}]
    }
)

gfile.SetContentFile(upload_file)
gfile.Upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=958497112279-8vd5d4ja0r4imq6ohad3mj9me5hhnb8p.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code&approval_prompt=force

Authentication successful.


In [13]:
gauth = GoogleAuth()
# Try to load saved client credentials
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    # Authenticate if they're not there
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    # Refresh them if expired
    gauth.Refresh()
else:
    # Initialize the saved creds
    gauth.Authorize()
# Save the current credentials to a file
gauth.SaveCredentialsFile("mycreds.txt")

drive = GoogleDrive(gauth)

gfile = drive.CreateFile(
    {
        'title': filename,
        'parents': [{'id': key_id}]
    }
)

gfile.SetContentFile(upload_file)
gfile.Upload()

KeyError: '_module'